In [ ]:
# (Tuỳ chọn) Cho Colab/Kaggle khi cần ép phiên bản:
# Ghi chú: Nếu runtime đã có torch 2.x phù hợp thì có thể bỏ qua.

# GPU (CUDA 12.1):
# %pip install -q --index-url https://download.pytorch.org/whl/cu121 \
#     torch==2.2.* torchvision==0.17.* torchaudio==2.2.*

# CPU only:
# %pip install -q --index-url https://download.pytorch.org/whl/cpu \
#     torch==2.2.* torchvision==0.17.* torchaudio==2.2.*

# Thư viện phụ trợ (khớp requirements.txt):
# %pip install -q torchtext==0.17.* torchdata==0.7.* spacy==3.7.* altair==5.* GPUtil sacrebleu

# (Tuỳ chọn) Tải model spaCy tiếng Anh/Đức nếu notebook này dùng tokenizer spaCy:
# %python -m spacy download en_core_web_sm
# %python -m spacy download de_core_news_sm


In [1]:
import os
from os.path import exists
import math
import copy
import time
import warnings

import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad  # Giữ nguyên API các cell sau đang dùng
from torch.optim.lr_scheduler import LambdaLR

import pandas as pd
import altair as alt

# torchtext/torchdata (giữ nguyên util đang dùng, dù bản mới khuyên DataPipes)
from torchtext.data.functional import to_map_style_dataset
import torchtext.datasets as datasets
from torchtext.vocab import build_vocab_from_iterator

from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler

import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

import spacy
import GPUtil

# Thiết lập chung
warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

# (Tuỳ chọn) Cảnh báo phiên bản để nhắc nâng cấp code về sau:
try:
    from importlib.metadata import version
    _tv = version("torchtext")
    _td = version("torchdata")
    _pt = torch.__version__
    print(f"[ENV] torch={_pt} | torchtext={_tv} | torchdata={_td}")
except Exception:
    pass


[ENV] torch=2.2.2+cpu | torchtext=0.17.2 | torchdata=0.7.1
